In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [2]:
import os
import sys

sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
from mle_toolbox import load_result_logs
from mle_toolbox.utils import load_meta_log
from utils.helper import get_best_roi_models, zip, save_dict

# Load logs and create submission

In [9]:
#experiment_dir = "../experiments/roi/elastic_resnet50_bo_50"
#experiment_dir = "../experiments/wb/elastic_resnet50_bo_15"

#experiment_dir = "../experiments/roi/elastic_efficientnet_b3_bo_50"
#experiment_dir = "../experiments/wb/elastic_efficientnet_b3_bo_50"

#experiment_dir = "../experiments/roi/elastic_resnext50_32x4d_bo_50"
#experiment_dir = "../experiments/wb/elastic_resnext50_32x4d_bo_50"
#experiment_dir = "../data/experiments/roi/mlp_efficientnet_b3_bo_50"
#experiment_dir = "../data/experiments/wb/mlp_efficientnet_b3_bo_30"
#experiment_dir = "../data/experiments/roi/elastic_vone_resnet50_bo_50"
#experiment_dir = "../data/experiments/wb/elastic_vone_resnet50_bo_30"
#experiment_dir = "../data/experiments/roi/elastic_vone_resnet50_at_bo_50"
#experiment_dir = "../data/experiments/wb/elastic_vone_resnet50_at_bo_30"

#experiment_dir = "../data/experiments/roi/elastic_vone_cornets_bo_50"
experiment_dir = "../data/experiments/wb/elastic_vone_cornets_bo_30"

meta_log, hyper_log = load_result_logs(experiment_dir)

In [10]:
meta_log['b_1_eval_0'].meta.experiment_dir

'experiments/wb/elastic_vone_cornets_bo_30/21-06-14_b_1_eval_0/'

# Get submission ready based on `experiment_dir`

In [13]:
track = "full_track"
if track == 'full_track': ROIs = ['WB']
else: ROIs = ['LOC','FFA','STS','EBA','PPA','V1','V2','V3','V4']

all_subjects = ['sub01', 'sub02', 'sub03', 'sub04', 'sub05',
                'sub06', 'sub07', 'sub08', 'sub09', 'sub10']

# Load in individual test npy files (subject/roi)
results = {}
for ROI in ROIs:
    ROI_results = {}
    for sub in all_subjects:
        hyper_sub = hyper_log[hyper_log.subject_id == sub]
        hyper_sub_roi = hyper_sub[hyper_sub.roi_type == ROI]
        # experiment_dir = hyper_sub_roi.experiment_dir.iloc[0]
        eval_id = hyper_sub_roi.run_id.iloc[0]
        experiment_dir = meta_log[eval_id].meta.experiment_dir
        mypath = os.path.join("../data/" + experiment_dir, "extra")
        try:
            sub_roi_layer = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
            ROI_result = np.load(mypath + "/" + sub_roi_layer[0])
            ROI_results[sub] = ROI_result
        except:
            print(mypath)
            continue
    results[ROI] = ROI_results

In [14]:
print(results.keys())
print(results['WB'].keys())
print(results['WB']['sub01'].shape)

dict_keys(['WB'])
dict_keys(['sub01', 'sub02', 'sub03', 'sub04', 'sub05', 'sub06', 'sub07', 'sub08', 'sub09', 'sub10'])
(102, 18222)


In [15]:
# Create model_submission sub directory
model_name = "2021-06-15_elastic_vone_cornets_bo_50"
sub_dir = f"../data/submissions/{model_name}/{track}/"
if not os.path.exists(sub_dir):
    os.makedirs(sub_dir)
    
save_dict(results, sub_dir + track + ".pkl")
zip(sub_dir, sub_dir + track + ".zip")

# Manual Training of Final Predictor Networks

In [8]:
from utils.helper import get_encoding_data
from encoding_models import EncoderFitter
import shutil

config_keys = ['dropout', 'hidden_act', 'learning_rate', 'num_hidden_layers',
               'num_hidden_units', 'optimizer_class', 'w_decay']

for e_id in meta_log.eval_ids:
    best_id = np.argmax(meta_log[e_id].stats.best_bo_score.mean)
    best_config = {}
    for c_key in config_keys:
        if type(meta_log[e_id].stats[c_key]) == list:
            config_value = meta_log[e_id].stats[c_key][0][best_id].decode()
        else:
            config_value = meta_log[e_id].stats[c_key].mean[best_id]
        best_config[c_key] = config_value
    best_layer = meta_log[e_id].stats['layer_id'][0][best_id].decode()
    #print(model_config, layer_id)
    
    subject_id = hyper_log[hyper_log.run_id == e_id].subject_id.iloc[0]
    roi_type = hyper_log[hyper_log.run_id == e_id].roi_type.iloc[0]
    
    X, y, X_test = get_encoding_data(fmri_dir="../data/participants_data_v2021",
                                     activations_dir='../data/features/alexnet/pca_100',
                                     layer_id=best_layer,
                                     subject_id=subject_id,
                                     roi_type=roi_type)
    fitter = EncoderFitter("mlp_network", 10,
                           X, y, X_test)
    model_params, y_pred = fitter.predict_on_test(best_config)
    dirpath = "../" + meta_log[e_id].meta.experiment_dir + "extra/"
    save_fname = dirpath + subject_id + "_" + roi_type + "_" + best_layer + "_test.npy"
    if os.path.exists(dirpath) and os.path.isdir(dirpath):
        shutil.rmtree(dirpath)
    os.makedirs(dirpath)
    np.save(save_fname , y_pred)
    print(e_id, save_fname)

KeyError: 'dropout'

# Construct ROI Submission from Invidually Best Results

In [ ]:
roi_paths = get_best_roi_models('../best_roi_models.json', prepend="../data/")

In [ ]:
roi_paths

In [ ]:
# ROI - Subject ID

ROIs = ['LOC', 'FFA','STS','EBA','PPA','V1','V2','V3','V4']

all_subjects = ['sub01', 'sub02', 'sub03', 'sub04', 'sub05',
                'sub06', 'sub07', 'sub08', 'sub09', 'sub10']

In [ ]:
results = {}
for ROI in ROIs:
    print(ROI)
    experiment_dir = roi_paths[ROI]["path"]
    meta_log, hyper_log = load_result_logs(experiment_dir)
    ROI_results = {}
    for sub in all_subjects:
        hyper_sub = hyper_log[hyper_log.subject_id == sub]
        hyper_sub_roi = hyper_sub[hyper_sub.roi_type == ROI]
        # experiment_dir = hyper_sub_roi.experiment_dir.iloc[0]
        eval_id = hyper_sub_roi.run_id.iloc[0]
        experiment_dir = meta_log[eval_id].meta.experiment_dir
        mypath = os.path.join("../data/" + experiment_dir, "extra")
        try:
            sub_roi_layer = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
            ROI_result = np.load(mypath + "/" + sub_roi_layer[0])
            ROI_results[sub] = ROI_result
        except:
            print(mypath)
            continue
    results[ROI] = ROI_results    

In [ ]:
# Create model_submission sub directory
model_name = "2021-06-15_combined"
sub_dir = f"../data/submissions/{model_name}/mini-track/"
if not os.path.exists(sub_dir):
    os.makedirs(sub_dir)
    
save_dict(results, sub_dir + track + ".pkl")
zip(sub_dir, sub_dir + track + ".zip")